[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Spinkk/TeachingTensorflow/blob/main/RNNs/Implementing%20Custom%20RNNs%20using%20Tensorflow%20and%20Keras.ipynb)

# Implementing Custom RNNs using Tensorflow and Keras

In this notebook, I show you how a basic RNN that uses a simple recurrent cell (no LSTM or GRU) transforms input by applying the same matrix multiplications at each time step, using the latest output as one of the inputs. After having investigated the computations involved in recurrent neural networks, you will also see how you can write your own RNN wrapper and simple rnn cell layers in tensorflow, which you can use as an orientation for the homework.


Remember that tf.keras.layers.Dense layers perform a matrix multiplication on their input, so we use these layers to define our RNN cell.

A basic RNN takes its own current hidden state (which is a vector of dimension hidden_dim for each example in the batch) and linearly transforms it with a weight matrix, creating a vector of n_outputs dimensions. In addition (quite literally) to that, the RNN cell takes the input at the current time-step t (which is the feature vector in the example for time-step t) and transforms it with another matrix multiplication to the same dimensionality of the hidden state. Adding the results from the two matrix multiplications together with a bias (and applying a tanh activation to the sum), we obtain the hidden state of the RNN-cell that will then be used at the next time-step t+1, together with the input data feature vector at t+1.

More formally written, the computation of a simple RNN cell at one time-step can be described as follows:

### $$h^{<t+1>} = tanh ( W_{hh} h^{<t>} + W_{xh} x^{<t>} + b )$$ 

Where $x^{<t>}$ is the feature input at time-step t, and $W_{xh}$ is a matrix of shape x (the dimensionality of the input feature vector at a single time-step) by h (the dimensionality of the hidden state).


Below we implement this type of an RNN on some input (not inside a tf.keras.Model or tf.keras.Layer but as separate layers since we don't do any model fitting here). First we show how to use tensorflow/keras inbuilt RNN cell and RNN wrapper layers to achieve this. By the end of the notebook you'll know how to implement these layers yourself from scratch.

Note: You will see the context defined by "with tf.device('/device:cpu:0'):" quite a lot in this notebook. This is to force tensorflow to do computations on the device specified. Since we do not need a GPU for this notebook, it's better to not have tensorflow reserve the GPU memory.

In [1]:
import tensorflow as tf

In [2]:
with tf.device('/device:cpu:0'):
    
    # a batch with 6 time steps of 16 features. The time-dimension is the second after the batch dimension.
    input_shape = (2, 6, 16)

    # The length of the resulting vector (similar to the units argument in Dense layers)
    hidden_dim = 3
    
    input_sequence = tf.random.uniform(shape = input_shape)
    
    simple_RNN_cell = tf.keras.layers.SimpleRNNCell(hidden_dim)
    
    # return_sequences=False means we only output the final hidden_state.
    RNN = tf.keras.layers.RNN(simple_RNN_cell, return_sequences=False)
    
    output = RNN(input_sequence)
    
    print(f"The output of the RNN, which is the last hidden state is \n{output}\n\n")
    

The output of the RNN, which is the last hidden state is 
[[-0.55842763  0.7973356   0.19838   ]
 [ 0.5054061   0.58539706 -0.84264874]]




# tf.keras.layers.RNN and tf.keras.layers.SimpleRNNCell from scratch

Now we want to look at how we can implement what happens inside the two pre-defined layers that we have used above.

To do so, we create two dense layers carrying the matrices $W_{hh}$ and $W_{xh}$ that are involved in the simple RNN cell. For the bias $b$ we create a separate tf.variable. Notice that we disable the use of a bias in the dense layers and do not use an activation either, because we want a simple matrix multiplication.

With the weight matrices and the bias defined, we can start to iterate over the same input sequence data that we have used before. The initial hidden state of the RNN cell is set to be a vector of zeros before looping over the input sequence.

At each time-step we use the previous hidden-state of the cell and update it, following the equation shown in the beginning of the notebook. The output of this custom RNN is then just the final hidden-state vector.

Notice how we use the same two weight matrices and the same bias for different temporal parts of the input data. This is why RNNs can be thought of as another variant of weight sharing.

In [3]:
with tf.device('/device:cpu:0'):
    ### We've seen the output of the randomly initialized simple RNN.
    ### Now we want to show the computations that were involved by doing the same with dense layers 
    
    ### Remember that dense layers without activation are just matrix multiplication of a weight matrix with input
    
    # We create the Dense layers (that will store the matrices)
    dense_layer_hstate = tf.keras.layers.Dense(hidden_dim, activation=None, use_bias=False)
    dense_layer_input = tf.keras.layers.Dense(hidden_dim, activation=None, use_bias=False)
    bias = tf.Variable(tf.zeros(hidden_dim, dtype=tf.float32), name="biases")
    
    # create the initial hidden state
    state = tf.zeros((1,hidden_dim), tf.float32)
    
    # iterate over the time-steps
    for t in tf.range(input_shape[1]):
        # on this iteration we use the input at timestep t
        input_t =input_sequence[:,t,:]
        
        # we compute the sum of the input at t matrix multiplied, with the previous state matrix multiplied
        # and an additional bias added.
        x_sum = dense_layer_input(input_t) + dense_layer_hstate(state) + bias
        
        # finally we use hyperbolic tangent as an activation function and update the RNN cell state
        state = tf.nn.tanh(x_sum)

    print(f"The output of the custom RNN that we built with dense layers is: \n{state}\n\n")

The output of the custom RNN that we built with dense layers is: 
[[-0.03773887  0.18126507 -0.47590804]
 [-0.7169903  -0.26814744  0.1255381 ]]




# Copying the weights of the pre-defined RNN to our custom RNN to verify the implementation

To make sure that we do the same computations as in the pre-defined tf.keras.layers.SimpleRNNCell and the wrapper layer (which implements the for loop) tf.keras.layers.RNN, we take the weights that we have used before and assign them to the two dense layers and the bias. We observe that we now get the exact same output - our implementation is correct.

This type of testing can be helpful to verify your custom implementations of the computations involved in the layers.

In [4]:
with tf.device('/device:cpu:0'):
    # Now we want to copy the weights of this RNN to a fully connected model to obtain the same output
    
    RNN_cell_input_weights = RNN.trainable_variables[0]

    RNN_cell_hstate_weights = RNN.trainable_variables[1]
    
    RNN_cell_biases = RNN.trainable_variables[2]
    
    dense_layer_hstate.weights[0].assign(tf.reshape(RNN_cell_hstate_weights, dense_layer_hstate.weights[0].shape))
    dense_layer_input.weights[0].assign(tf.reshape(RNN_cell_input_weights, dense_layer_input.weights[0].shape))
    bias.assign(RNN_cell_biases)
    
    # again we run our custom RNN, this time with the same weights as the tf.keras.layers.RNN version
    
    # same code as above
    state = tf.zeros((1,hidden_dim), tf.float32)

    for t in tf.range(input_shape[1]):
        input_t =input_sequence[:,t,:]
        
        x_sum = dense_layer_input(input_t) + dense_layer_hstate(state) + bias

        state = tf.nn.tanh(x_sum)
    
    print(f"With the same weights as the pre-defined RNN, our custom RNN's output is\n {state}\n")
    
    print(f"The outputs of the pre-defined RNN and our custom RNN are the same: {tf.reduce_all(state==output)}")

With the same weights as the pre-defined RNN, our custom RNN's output is
 [[-0.55842763  0.7973356   0.19838   ]
 [ 0.5054061   0.58539706 -0.84264874]]

The outputs of the pre-defined RNN and our custom RNN are the same: True


# Return sequences of hidden-states with an RNN

Before we have set the return_sequences argument to False, and we ended up with a single vector as the output (the final hidden state) of the RNN. To obtain a sequence from the RNN, we want to return the hidden states from all the time-steps. This is useful for cases in which we want to translate from one sequence to another, like in language translation or audio-processing. To implement this with the pre-defined layers, we use the same simple rnn cell layer as before but re-instantiate the RNN wrapper layer with the new argument set.

In [5]:
with tf.device('/device:cpu:0'):
    # simple_RNN_cell and input_sequence remain unchanged
    
    # return_sequences=True means we output the hidden_states of all time-steps.
    RNN = tf.keras.layers.RNN(simple_RNN_cell, return_sequences=True)
    
    RNN_outputs = RNN(input_sequence)
    
    print(f"The output of the RNN, which is the last hidden state is \n {RNN_outputs}\n\n")

The output of the RNN, which is the last hidden state is 
 [[[ 0.78782976  0.6367441  -0.16496447]
  [-0.14419898  0.57446164 -0.48047882]
  [-0.40607536  0.6472315  -0.6515597 ]
  [-0.23524919  0.9488684  -0.6779934 ]
  [ 0.47880217  0.7325374  -0.7649211 ]
  [-0.55842763  0.7973356   0.19838   ]]

 [[ 0.6847383   0.7199698  -0.54862726]
  [ 0.35626864  0.9001869   0.00961799]
  [-0.05822116  0.65383255 -0.07338887]
  [ 0.01840258  0.2500062  -0.26280335]
  [ 0.6293602   0.5622283  -0.6971662 ]
  [ 0.5054061   0.58539706 -0.84264874]]]




# Return sequences from scratch

To implement this in a way that will allow you to also use this in tensorflow models that use the @tf.function decorator, we can't just create a list of the hidden states and append to it - tensorflow does not allow list appends in graph mode. Instead we can make use of a tf.TensorArray object (this also allows for the case in which we do not know how many time-steps we will process and thus can't know beforehand how many hidden-states we want to output).

A tf.TensorArray object is not just a tensor and thus we need to call the .stack() method on it to obtain the hidden_states that it stores. Since the result has the batch-dimension and the time-dimension switched, we make use of a permuted transpose.

In [6]:
with tf.device('/device:cpu:0'):

    state = tf.zeros((input_sequence.shape[0], hidden_dim), tf.float32)

    # initialize the hidden_states TensorArray that we want to output (shape is: batch, time-steps, h_dim)
    hidden_states = tf.TensorArray(dtype=tf.float32, size = input_sequence.shape[1])

    for t in tf.range(input_shape[1]):
        input_t =input_sequence[:,t,:]

        x_sum = dense_layer_input(input_t) + dense_layer_hstate(state) + bias

        state = tf.nn.tanh(x_sum)

        # write the states to the TensorArray
        hidden_states = hidden_states.write(t, state)

    # transpose the sequence of hidden_states from TensorArray accordingly (batch and time dimensions switched)
    custom_RNN_outputs = tf.transpose(hidden_states.stack(), [1,0,2])

    print(f"The output of the custom RNN that we built with dense layers is: \n{custom_RNN_outputs}\n\n")

The output of the custom RNN that we built with dense layers is: 
[[[ 0.78782976  0.6367441  -0.16496447]
  [-0.14419898  0.57446164 -0.48047882]
  [-0.40607536  0.6472315  -0.6515597 ]
  [-0.23524919  0.9488684  -0.6779934 ]
  [ 0.47880217  0.7325374  -0.7649211 ]
  [-0.55842763  0.7973356   0.19838   ]]

 [[ 0.6847383   0.7199698  -0.54862726]
  [ 0.35626864  0.9001869   0.00961799]
  [-0.05822116  0.65383255 -0.07338887]
  [ 0.01840258  0.2500062  -0.26280335]
  [ 0.6293602   0.5622283  -0.6971662 ]
  [ 0.5054061   0.58539706 -0.84264874]]]




Now let us verify that the @tf.function decorator works with this.

In [7]:
with tf.device('/device:cpu:0'):
    @tf.function
    def tf_func():

        state = tf.zeros((input_sequence.shape[0], hidden_dim), tf.float32)

        # initialize the hidden_states TensorArray that we want to output (shape is: batch, time-steps, h_dim)
        hidden_states = tf.TensorArray(dtype=tf.float32, size=input_sequence.shape[1])

        for t in tf.range(input_shape[1]):
            input_t =input_sequence[:,t,:]

            x_sum = dense_layer_input(input_t) + dense_layer_hstate(state) + bias

            state = tf.nn.tanh(x_sum)

            # write the states to the TensorArray
            hidden_states = hidden_states.write(t, state)

        # transpose the sequence of hidden_states from TensorArray accordingly (batch and time dimensions switched)
        custom_RNN_outputs = tf.transpose(hidden_states.stack(), [1,0,2])
        
        return custom_RNN_outputs
    custom_RNN_outputs = tf_func()
    print(f"The output of the custom RNN that we built with dense layers is: \n{custom_RNN_outputs}\n\n")

The output of the custom RNN that we built with dense layers is: 
[[[ 0.78782976  0.6367441  -0.16496447]
  [-0.14419898  0.57446164 -0.48047882]
  [-0.40607536  0.6472315  -0.6515597 ]
  [-0.23524919  0.9488684  -0.6779934 ]
  [ 0.47880217  0.7325374  -0.7649211 ]
  [-0.55842763  0.7973356   0.19838   ]]

 [[ 0.6847383   0.7199698  -0.54862726]
  [ 0.35626864  0.9001869   0.00961799]
  [-0.05822116  0.65383255 -0.07338887]
  [ 0.01840258  0.2500062  -0.26280335]
  [ 0.6293602   0.5622283  -0.6971662 ]
  [ 0.5054061   0.58539706 -0.84264874]]]




Finally, we again verify that the outputs with return_sequences=True match between our custom implementation of the computations involved and the pre-defined RNN-wrapper and cell. Since we did not re-initialize the weights, the outputs should still be the same.

In [8]:
print(tf.reduce_all(custom_RNN_outputs == RNN_outputs).numpy())

True


# Implementing custom RNNs with subclassing

Let's start with writing the RNNWrapper layer which loops over the observations at the different time-steps.

Like in tf.keras.layers.RNN, we implement a return_sequences argument that controls whether to return a sequence of hidden states or just the last hidden state.

(!) Important: This RNNWrapper is written only with the Simple RNN Cell in mind. It does not (yet - it is part of your task for the homework to adjust this) allow for LSTM Cells because here we only take care of the hidden_states but not of the cell state (since it does not exist for the basic rnn).

In [9]:
class RNNWrapper(tf.keras.layers.Layer):
    def __init__(self, RNN_Cell, return_sequences=False):
        super(RNNWrapper, self).__init__()
        
        self.return_sequences = return_sequences
        
        self.cell = RNN_Cell

    def call(self, data, training=False):
        
        length = data.shape[1]
        
        # initialize state of the simple rnn cell
        state = tf.zeros((data.shape[0], self.cell.units), tf.float32)
        
        # initialize array for hidden states (only relevant if self.return_sequences == True)
        hidden_states = tf.TensorArray(dtype=tf.float32, size=length)

        for t in tf.range(length):
            input_t = data[:,t,:]

            state = self.cell(input_t, state, training)

            if self.return_sequences:
                # write the states to the TensorArray
                #hidden_states = hidden_states.write(t, state)
                hidden_states.append(state)
        
        if self.return_sequences:
            # transpose the sequence of hidden_states from TensorArray accordingly 
            #(batch and time dimensions are otherwise switched after .stack())
            outputs = tf.transpose(hidden_states.stack(), [1,0,2])
        
        else:
            # take the last hidden state of the simple rnn cell
            outputs = state
        
        return outputs

Next we implement the SimpleRNNCell which is compatible with our custom RNN wrapper layer, because it only has a single hidden state (no cell state like the LSTM).

Here we 

In [10]:
class CustomSimpleRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, kernel_regularizer=None ):
        super(CustomSimpleRNNCell, self).__init__()
        
        self.units = units
        
        self.dense_hstate = tf.keras.layers.Dense(units, 
                                                  kernel_regularizer=kernel_regularizer, 
                                                  use_bias=False)
        
        self.dense_input = tf.keras.layers.Dense(units, 
                                                 kernel_regularizer=kernel_regularizer, 
                                                 use_bias=False)
        
        self.bias = tf.Variable(tf.zeros(units), name="RNN_Cell_biases")
        
        self.state_size = units
        
        
    def call(self, input_t, state,training=False):
        
        # we compute the sum of the input at t matrix multiplied and the previous state matrix multiplied
        # and an additional bias added.
        x_sum = self.dense_input(input_t) + self.dense_hstate(state) + self.bias
        
        # finally we use hyperbolic tangent as an activation function to update the RNN cell state
        state = tf.nn.tanh(x_sum)
        
        return (state)

Next, we define an RNN_model class, which uses the simple RNN to obtain a single feature vector from the time-series and makes a prediction based on this feature vector, using two fully connected layers.

In [11]:
class RNN_Model(tf.keras.Model):
    def __init__(self, units):
        super(RNN_Model, self).__init__()

        self.RNNWrapper = RNNWrapper(CustomSimpleRNNCell(units), return_sequences=False)
        
        self.dense = tf.keras.layers.Dense(128, activation="relu")
        
        self.out = tf.keras.layers.Dense(10, activation="softmax")
        
    #@tf.function(experimental_relax_shapes=True)
    def call(self, data, training=False):
        
        x = self.RNNWrapper(data, training)
        x = self.dense(x)
        x = self.out(x)
        
        return x

Lastly, we instantiate and use an RNN model on some random input as before.

In [12]:
with tf.device('/device:cpu:0'):
    input_sequence = tf.random.uniform(shape = (2,24,16), dtype=tf.float32)

    hidden_state_size = 3

    rnn = RNN_Model(hidden_state_size)
    prediction = rnn(input_sequence)
    tf.print(f"The randomly initialized RNN model outputs the class probabilities \n{prediction}\n for the random input.")

The randomly initialized RNN model outputs the class probabilities 
[[0.09028652 0.09195881 0.09046976 0.10634772 0.12041804 0.11259319
  0.10579    0.08614301 0.09627467 0.0997183 ]
 [0.09764748 0.09690416 0.09158446 0.11423371 0.09512214 0.10587687
  0.09874821 0.08770555 0.10301013 0.10916731]]
 for the random input.


As a last thing, we can verify our RNN can take inputs of different lengths - one of the major advantages of using RNNs!

In [13]:
with tf.device('/device:cpu:0'):
    
    print(f"Previous input shape was: {input_sequence.shape}\n\n")
    
    input_sequence = tf.random.uniform(shape = (8,64,16), dtype=tf.float32)
    
    print(f"New input shape is: {input_sequence.shape}\n\n")
    
    print("We see that both the batch size and the sequence length are different and the RNN still works.\n\
This would not have worked with an MLP model that has different weights for each temporal part of the input\
sequence. \n\n")

    # we re-use the same rnn model instantiation.
    prediction = rnn(input_sequence)
    tf.print(f"We obtain predictions of shape {prediction.shape} since we use a classification head with 10 units.")

Previous input shape was: (2, 24, 16)


New input shape is: (8, 64, 16)


We see that both the batch size and the sequence length are different and the RNN still works.
This would not have worked with an MLP model that has different weights for each temporal part of the inputsequence. 


We obtain predictions of shape (8, 10) since we use a classification head with 10 units.


In [14]:
rnn.summary()

Model: "rnn__model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rnn_wrapper (RNNWrapper)    multiple                  60        
                                                                 
 dense_4 (Dense)             multiple                  512       
                                                                 
 dense_5 (Dense)             multiple                  1290      
                                                                 
Total params: 1,862
Trainable params: 1,862
Non-trainable params: 0
_________________________________________________________________


## **(Optional. This is not allowed for the homework)** The same model can also be defined with pre-defined layers and using the Functional API, which shows a nicer summary including output shapes.

In [19]:
with tf.device('/device:cpu:0'):

    # a keras input which will be used to track computations done by the layers.
    RNN_pseudo_input = tf.keras.Input(shape=(None,16), batch_size=None)
    
    # RNN feature extractor of the model using the layers we've defined
    x = tf.keras.layers.RNN(tf.keras.layers.SimpleRNNCell(3), return_sequences=False)(RNN_pseudo_input)
        
    # classification head of the model
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    x_out = tf.keras.layers.Dense(10, activation="softmax")(x)
    
    # instantiate model with functional api, tracing computations and layers between RNN_input and x_out
    functional_model = tf.keras.Model(RNN_pseudo_input, x_out)

    # input shape
    input_sequence = tf.random.uniform(shape = (1,24,16), dtype=tf.float32)
    
    # call the functional model on the input sequence from before
    prediction = functional_model(input_sequence)
    
    tf.print(prediction)
    
    functional_model.summary()

[[0.0749586448 0.0989045724 0.11350774 ... 0.0843760297 0.093167223 0.108546838]]
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, 16)]        0         
                                                                 
 rnn_6 (RNN)                 (None, 3)                 60        
                                                                 
 dense_14 (Dense)            (None, 128)               512       
                                                                 
 dense_15 (Dense)            (None, 10)                1290      
                                                                 
Total params: 1,862
Trainable params: 1,862
Non-trainable params: 0
_________________________________________________________________
